In [1]:
from minio import Minio
import s3fs
import os
import matplotlib.pyplot as plt
import xarray as xr
import pickle
import numpy as np
from sympl import get_constant
import metpy
from metpy.units import units

fs = s3fs.S3FileSystem(
    client_kwargs={
        "endpoint_url": "http://192.168.1.237:9000"
    },
    key="d0d250b2541ac33f4660",
    secret="2fb32d964768bc94a3c0"
)
output_dir='/home/scratch/Abel_data/hw_processed/ctrl/'


Rd = get_constant('gas_constant_of_dry_air', 'J kg^-1 K^-1')
Cp =\
    get_constant('heat_capacity_of_dry_air_at_constant_pressure',
                 'J kg^-1 K^-1')
g=get_constant('gravitational_acceleration',
                 'm s^-2') 

In [85]:
dp=2000
Lx=np.radians(2.8125)*6371*1000
Ly=np.radians(2.79)*6371*1000

# climfile=output_dir+'climatology.nc'
# clim=xr.open_dataset(climfile)
# p1=clim['air_pressure']
# p2=p1.copy()
# p2[1:,:]=p1[:-1,:].values[:]
# # print(p2, clim['surface_air_pressure'])
# p2[0,:]=clim['surface_air_pressure'].values[:]

# lat_rad=np.radians(clim['air_temperature'].lat)
# z=(Rd*clim['air_temperature']/g*np.log(p2/p1)).cumsum(dim='lev')
# dse_clim=Cp*clim['air_temperature']+g*z
# u_clim=clim['eastward_wind']; v_clim=clim['northward_wind']; w_clim=clim['vertical_wind']

def avg_cons(x):
    return (x.values[1:]+x.values[:-1])/2

def diff_cons(x):
    return (x.values[1:]-x.values[:-1])

def compute_dse(ds, latc, lonc):
    
    p1=ds['air_pressure']
    p2=p1.copy()
    p2[:,1:]=p1[:,:-1].values[:]
    p2[:,0]=ds['surface_air_pressure'].values[:]
    
    lat_rad=np.radians(ds['air_temperature'].lat)
    
    z=(Rd*ds['air_temperature']/g*np.log(p2/p1)).cumsum(dim='lev')
    dse=((Cp*ds['air_temperature']+g*z)*dp/g*np.cos(lat_rad)*Lx*Ly).sel({'lat':slice(latc+5,latc-5), 'lon':slice(lonc-5,lonc+5),
                                                                         'lev':slice(98000, 84000)})
    return dse.sum(dim=('lat', 'lon','lev'))

def compute_dse_tend(ds, latc, lonc):
    
    ds=ds['air_temperature'].sel({'lat':slice(latc+5,latc-5), 'lon':slice(lonc-5,lonc+5),'lev':slice(90000, 90000)}).mean(dim='lev')
    
    t1=metpy.calc.first_derivative(ds, axis='lon')
    
    
    lat=(ds.lat); lon=(ds.lon)
    lon_rad=np.radians(lon.values[:]); lat_rad=np.radians(lat.values[:]) 

    dx=np.radians((lon).diff(dim='lon'))*6378137*units.metre*np.cos(np.radians(lat))
    
#     print(ds.diff(dim='lon'))
#     print(dx.T)
    
    t2=metpy.calc.first_derivative(ds, axis=1, delta=dx.T)
    
#     print(t1)
    print(t2)
    
#     print(ds)
    ds=ds.values[:]
#     print(ds)
    
#     print(ds[:,1:]-ds[:,:-1])
#     print((lon_rad[np.newaxis, 1:]-lon_rad[np.newaxis, :-1])*6378137*np.cos(lat_rad[:, np.newaxis]))
    t3=(ds[1:-1,2:]-ds[1:-1,:-2])/((lon_rad[np.newaxis, 2:]-lon_rad[np.newaxis, :-2])*6378137*np.cos(lat_rad[1:-1]))
    print(t3)
    
#     t1.plot()
#     plt.show()
#     print(t2)
#     plt.show()
    
    return 0
                                                                                                
    
latc=50
output_file=output_dir+'hw_temp_'+str(latc)
with open(output_file, 'rb') as f:
    hw_master=pickle.load(f)
    
hw_index=1

y=1
loop=0
dse_master={'index':[], 'T':[], 'lonc':[], 'dse_low':[], 'intensity':[], 'duration':[], 'maximum':[],
            'dseres':[],'dsez':[],'dsex':[],'dsey':[], 'dsetot':[]}

full_length=1
# while hw_index<=len(hw_master['index']):
while hw_index<=full_length:
    r=hw_master['run'][hw_index-1]
    zarr_path = "abel-long-run/Regridded_data/run"+str(r)+"/year"+str(y)
    ds = xr.open_zarr(fs.get_mapper(zarr_path))

    time_slice=hw_master['time'][hw_index-1]
    ds_hw=ds.sel({'time':time_slice[0]})
    print(hw_index)

#             dse_low=compute_dse(ds_hw, latc, hw_master['lonc'][hw_index-1])
    dse_tend_low=compute_dse_tend(ds_hw, latc, hw_master['lonc'][hw_index-1])

    hw_index=hw_index+1
    break
            
# print(dse_master)

1
[[-8.309844386469346e-06 -8.347228965848688e-06 -8.21167581485237e-06  -7.903184933480498e-06] [-7.405860185603452e-06 -6.5286286855988085e-06 -5.5423879013228065e-06  -4.447137832774579e-06] [-6.50694965169125e-06 -5.367059573771092e-06 -4.222684704608998e-06  -3.073825044205402e-06] [-7.022997121406685e-06 -5.881698683187376e-06 -4.899845777479026e-06  -4.077438404281094e-06]] / meter
[[-6.52862869e-06 -5.22721359e-06]
 [-5.69066588e-06 -4.22268470e-06]]
